In [104]:
import pandas as pd

### 1. Чтение данных
Читаем данные, удаляем ненужные колонки

In [105]:
data = pd.read_csv('./data-001.csv',
                   parse_dates=['date_time'])
data = data.drop(
    columns=['campaign_clicks', 'oaid_hash'])
data = data.sort_values(by='date_time')
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
1390198,2021-09-01 00:02:49,30,596,0,7,596,0.050,0.064560,-4.312062,603,0.050,0.056510,-4.370191,1,0
5041415,2021-09-26 00:00:00,41,29,3,0,29,0.002,0.016386,-4.736584,6,0.002,0.020875,-4.898257,1,0
1442602,2021-09-26 00:00:00,1,188,2,15,188,0.008,0.014186,-3.811444,11754925,0.330,0.012351,-2.106896,1,0
7232498,2021-09-26 00:00:00,17,52,2,5,52,0.008,0.013550,-4.317590,41,0.004,0.067812,-3.739501,1,0
14938691,2021-09-26 00:00:00,47,73,4,13,73,0.008,0.120974,-2.382508,1040,0.008,0.157515,-3.037939,1,0


### 2. Анализ данных
Проанализируем данные: статистики по каждой фиче, количество уникальных значений, отсутствующих значений, гистограммы распределений значений каждой из фич.

In [106]:
def analysis(data: pd.DataFrame):
    # множество статистических значений для датасета
    display(data.describe())

    # количество уникальных значений для каждой фичи
    print('Unique values count for each feature:')
    print(data.nunique(axis=0, dropna=True))
    print('-' * 60)

    # количество NaN значений в каждом столбце
    print('NaN values count for each feature:')
    print(data.isna().sum())
    print('-' * 60)

    # распределение количества сэмплов по дате и времени -- позволит увидеть, за какие моменты у нас в принципе есть данные, и в каком объеме
    print(f"Start day: {data['date_time'].min()}")
    print(f"End day: {data['date_time'].max()}")
    print("Date time samples count:")
    display(data.groupby('date_time').size())


In [107]:
analysis(data)

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
count,15821472,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,1.582147e+07,1.582140e+07,1.582140e+07,1.582140e+07,1.582147e+07,1.580173e+07,1.580173e+07,1.580173e+07,15821472.0,1.582147e+07
mean,2021-09-29 06:17:04.484696576,8.152679e+01,3.816483e+02,1.840605e+00,4.346986e+00,6.571314e+05,1.969248e-01,7.440764e-01,-3.979590e+00,2.605069e+06,1.415899e+00,5.485722e-01,-3.968883e+00,1.0,2.668835e-02
min,2021-09-01 00:02:49,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.763740e-02,-8.588970e+00,0.000000e+00,0.000000e+00,-6.983897e-02,-9.562188e+00,1.0,0.000000e+00
25%,2021-09-27 10:24:02,1.400000e+01,5.200000e+01,1.000000e+00,0.000000e+00,7.300000e+01,5.000000e-03,1.643856e-02,-4.515871e+00,9.900000e+01,4.000000e-03,1.615219e-02,-4.529519e+00,1.0,0.000000e+00
50%,2021-09-29 02:22:11,1.900000e+01,2.170000e+02,2.000000e+00,4.000000e+00,3.030000e+02,1.000000e-02,3.539307e-02,-3.921164e+00,4.600000e+02,1.400000e-02,3.556666e-02,-3.928674e+00,1.0,0.000000e+00
75%,2021-09-30 21:36:15,6.000000e+01,6.110000e+02,3.000000e+00,7.000000e+00,7.200000e+02,3.000000e-02,8.022935e-02,-3.421280e+00,1.236000e+03,5.000000e-02,7.546751e-02,-3.390867e+00,1.0,0.000000e+00
max,2021-10-02 23:59:59,3.443000e+03,1.632000e+03,1.000000e+01,1.600000e+01,1.146423e+07,1.000000e+02,6.910888e+02,3.149981e-01,1.462360e+07,1.000000e+02,6.910885e+02,4.756181e-01,1.0,1.000000e+00
std,NaN,1.632448e+02,3.959386e+02,1.530005e+00,4.317701e+00,2.606008e+06,2.733440e+00,1.670358e+01,1.143982e+00,5.230253e+06,8.689053e+00,1.419136e+01,1.186403e+00,0.0,1.611710e-01


Unique values count for each feature:
date_time        604712
zone_id            3444
banner_id          1633
os_id                11
country_id           17
banner_id0       946937
rate0               428
g0             15147522
coeff_sum0      5262825
banner_id1      3160859
rate1               845
g1             15169168
coeff_sum1      5660517
impressions           1
clicks                2
dtype: int64
------------------------------------------------------------
NaN values count for each feature:
date_time          0
zone_id            0
banner_id          0
os_id              0
country_id         0
banner_id0         0
rate0             69
g0                69
coeff_sum0        69
banner_id1         0
rate1          19744
g1             19744
coeff_sum1     19744
impressions        0
clicks             0
dtype: int64
------------------------------------------------------------
Start day: 2021-09-01 00:02:49
End day: 2021-10-02 23:59:59
Date time samples count:


date_time
2021-09-01 00:02:49     1
2021-09-26 00:00:00    28
2021-09-26 00:00:01    28
2021-09-26 00:00:02    25
2021-09-26 00:00:03    30
                       ..
2021-10-02 23:59:55    22
2021-10-02 23:59:56    22
2021-10-02 23:59:57    19
2021-10-02 23:59:58    20
2021-10-02 23:59:59    19
Length: 604712, dtype: int64

### 3. Фильтрация и конструирование фичей

Удалим повторяющиеся строки таблицы, если такие имеются

In [108]:
data = data.drop_duplicates()

Анализируя статистики, можем увидеть, что значение поля impressions всегда одно и равно единице. Значит, все баннеры-сэмплы, данные о которых у нас имеются, в каждой конфигурации были показаны по одному разу. Так как фича является константой, она не играет роли при выборе класса объекта в нашей задаче классификации. Можем обойтись без этой колонки

In [109]:
data = data.drop(columns=['impressions'])

Заметим, что для даты 2021-09-01 только одно наблюдение. Удалим данные за этот день из датасета как выброс.

In [110]:
data = data[data['date_time'] >= '2021-09-02']
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
5041415,2021-09-26,41,29,3,0,29,0.002,0.016386,-4.736584,6,0.002,0.020875,-4.898257,0
1442602,2021-09-26,1,188,2,15,188,0.008,0.014186,-3.811444,11754925,0.330,0.012351,-2.106896,0
7232498,2021-09-26,17,52,2,5,52,0.008,0.013550,-4.317590,41,0.004,0.067812,-3.739501,0
14938691,2021-09-26,47,73,4,13,73,0.008,0.120974,-2.382508,1040,0.008,0.157515,-3.037939,0
11536774,2021-09-26,48,266,0,1,266,0.005,0.064730,-3.774143,124,0.003,0.052319,-3.481748,0


Заметим, что все имеющиеся данные затрагивают только 2 месяца -- сентябрь и начало октября. Информация про год избыточна, тк у всех сэмплов одинакова. Кроме того, можем убрать измерения секунд, нам хватит знания часа и минут для каждого примера. Добавим отдельные столбцы day, hour, minute. После разделения на train и test впоследствии удалим столбец date_time.

In [111]:
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data['minutes'] = data['date_time'].dt.minute

В новых колонках rate, g, coeff_sum встречаются NaN. Удалим их.

In [112]:
data = data.dropna()

Отфильтруем случаи, когда banner_id0 не совпадает с banner_id.

In [113]:
data = data[data['banner_id0'] == data['banner_id']]

Добавим интеракции порядка 2 -- попарная конкатенация категориальных фич. Новая колонка интеракций будет иметь название вида 'feature1:feature2'. Значение интеракции будет являться значениями исходных колонок, сконкатенированными через двоеточие.

In [114]:
from typing import List

In [115]:
def add_row_interactions(df_row, interaction_features: List[str]):
    interactions_values = [f'{df_row[feature]}' for feature in interaction_features]
    return ':'.join(interactions_values)


def create_interaction_column(data: pd.DataFrame, interaction_features: List[str]):
    interaction_features_column = ':'.join(interaction_features)
    data[interaction_features_column] = data.apply(
        lambda row: add_row_interactions(row, interaction_features),
        axis=1
    )

In [118]:
categorial_columns = ['zone_id', 'banner_id', 'os_id', 'country_id']

for i in range(0, len(categorial_columns)):
    for j in range(i + 1, len(categorial_columns)):
        feature1 = categorial_columns[i]
        feature2 = categorial_columns[j]
        create_interaction_column(data, [feature1, feature2])

display(data)

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,...,clicks,day,hour,minutes,zone_id:banner_id,zone_id:os_id,zone_id:country_id,banner_id:os_id,banner_id:country_id,os_id:country_id
5041415,2021-09-26 00:00:00,41,29,3,0,29,0.0020,0.016386,-4.736584,6,...,0,26,0,0,41:29,41:3,41:0,29:3,29:0,3:0
1442602,2021-09-26 00:00:00,1,188,2,15,188,0.0080,0.014186,-3.811444,11754925,...,0,26,0,0,1:188,1:2,1:15,188:2,188:15,2:15
7232498,2021-09-26 00:00:00,17,52,2,5,52,0.0080,0.013550,-4.317590,41,...,0,26,0,0,17:52,17:2,17:5,52:2,52:5,2:5
14938691,2021-09-26 00:00:00,47,73,4,13,73,0.0080,0.120974,-2.382508,1040,...,0,26,0,0,47:73,47:4,47:13,73:4,73:13,4:13
11536774,2021-09-26 00:00:00,48,266,0,1,266,0.0050,0.064730,-3.774143,124,...,0,26,0,0,48:266,48:0,48:1,266:0,266:1,0:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11510884,2021-10-02 23:59:59,20,1236,2,0,1236,0.0460,0.009155,-6.085725,13765667,...,0,2,23,59,20:1236,20:2,20:0,1236:2,1236:0,2:0
10139863,2021-10-02 23:59:59,24,180,2,6,180,0.0080,0.038556,-3.791884,403,...,0,2,23,59,24:180,24:2,24:6,180:2,180:6,2:6
5597133,2021-10-02 23:59:59,73,92,1,0,92,0.0594,0.023820,-4.641030,12584620,...,0,2,23,59,73:92,73:1,73:0,92:1,92:0,1:0
8660907,2021-10-02 23:59:59,17,1235,4,0,1235,0.0670,0.027977,-5.617197,1240,...,0,2,23,59,17:1235,17:4,17:0,1235:4,1235:0,4:0


In [119]:
data.columns

Index(['date_time', 'zone_id', 'banner_id', 'os_id', 'country_id',
       'banner_id0', 'rate0', 'g0', 'coeff_sum0', 'banner_id1', 'rate1', 'g1',
       'coeff_sum1', 'clicks', 'day', 'hour', 'minutes', 'zone_id:banner_id',
       'zone_id:os_id', 'zone_id:country_id', 'banner_id:os_id',
       'banner_id:country_id', 'os_id:country_id'],
      dtype='object')

### 4. Train/ test splitting
Разделим данные на тренировочную и тестовую части. Тренировать модель будем на всех данных, полученных до последнего дня, тестировать -- на данных последнего дня.

In [120]:
# разделяющий день -- последний день, начиная с 00:00
splitting_datetime = data['date_time'].max()
splitting_datetime = splitting_datetime.replace(hour=0, minute=0, second=0)
print(f'Splitting datetime: {splitting_datetime}')

train_data = data[data['date_time'] < splitting_datetime]
test_data = data[data['date_time'] >= splitting_datetime]

train_data = train_data.drop(columns=['date_time'])
test_data = test_data.drop(columns=['date_time'])

print(f'Train samples: {len(train_data)}')
print(f'Test samples: {len(test_data)}')

display(train_data)
display(test_data)

Splitting datetime: 2021-10-02 00:00:00
Train samples: 12026120
Test samples: 1883622


,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,...,clicks,day,hour,minutes,zone_id:banner_id,zone_id:os_id,zone_id:country_id,banner_id:os_id,banner_id:country_id,os_id:country_id
5041415,41,29,3,0,29,0.002,0.016386,-4.736584,6,0.002,...,0,26,0,0,41:29,41:3,41:0,29:3,29:0,3:0
1442602,1,188,2,15,188,0.008,0.014186,-3.811444,11754925,0.330,...,0,26,0,0,1:188,1:2,1:15,188:2,188:15,2:15
7232498,17,52,2,5,52,0.008,0.013550,-4.317590,41,0.004,...,0,26,0,0,17:52,17:2,17:5,52:2,52:5,2:5
14938691,47,73,4,13,73,0.008,0.120974,-2.382508,1040,0.008,...,0,26,0,0,47:73,47:4,47:13,73:4,73:13,4:13
11536774,48,266,0,1,266,0.005,0.064730,-3.774143,124,0.003,...,0,26,0,0,48:266,48:0,48:1,266:0,266:1,0:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3594598,254,584,2,10,584,0.001,0.770310,-2.898877,168,0.001,...,0,1,23,59,254:584,254:2,254:10,584:2,584:10,2:10
8604214,34,47,2,5,47,0.100,0.043624,-3.635624,687,0.100,...,0,1,23,59,34:47,34:2,34:5,47:2,47:5,2:5
1633515,3,1239,2,0,1239,0.046,0.008633,-5.696672,1236,0.046,...,0,1,23,59,3:1239,3:2,3:0,1239:2,1239:0,2:0
6571049,139,49,0,0,49,0.014,0.019515,-3.590981,21,0.014,...,0,1,23,59,139:49,139:0,139:0,49:0,49:0,0:0


,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,...,clicks,day,hour,minutes,zone_id:banner_id,zone_id:os_id,zone_id:country_id,banner_id:os_id,banner_id:country_id,os_id:country_id
14196412,14,1239,1,0,1239,0.0460,0.011367,-6.356145,1234,0.04600,...,0,2,0,0,14:1239,14:1,14:0,1239:1,1239:0,1:0
8706638,525,174,3,0,174,0.0010,0.063288,-3.107591,104,0.00100,...,0,2,0,0,525:174,525:3,525:0,174:3,174:0,3:0
13000378,14,175,2,9,175,0.0070,0.042959,-3.023266,232,0.00700,...,0,2,0,0,14:175,14:2,14:9,175:2,175:9,2:9
9767447,0,76,1,3,76,0.0080,0.051014,-4.665202,34,0.00873,...,0,2,0,0,0:76,0:1,0:3,76:1,76:3,1:3
9054327,24,428,1,10,428,0.0010,0.157651,-3.672826,719,0.00100,...,0,2,0,0,24:428,24:1,24:10,428:1,428:10,1:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11510884,20,1236,2,0,1236,0.0460,0.009155,-6.085725,13765667,0.50000,...,0,2,23,59,20:1236,20:2,20:0,1236:2,1236:0,2:0
10139863,24,180,2,6,180,0.0080,0.038556,-3.791884,403,0.00800,...,0,2,23,59,24:180,24:2,24:6,180:2,180:6,2:6
5597133,73,92,1,0,92,0.0594,0.023820,-4.641030,12584620,1.50000,...,0,2,23,59,73:92,73:1,73:0,92:1,92:0,1:0
8660907,17,1235,4,0,1235,0.0670,0.027977,-5.617197,1240,0.06700,...,0,2,23,59,17:1235,17:4,17:0,1235:4,1235:0,4:0


Разделим train и test данные на признаки X и таргет y. Отдельно выделим тестовые данные X_test_0 с banner_id = banner_id0 и X_test_1 для случая banner_id = banner_id1.

In [121]:
def split_to_X_y(data: pd.DataFrame):
    X = data.drop(columns=['clicks'])
    y = data['clicks']
    return X, y


X_train, y_train = split_to_X_y(train_data)
X_test, y_test = split_to_X_y(test_data)

X_test_0 = X_test.copy()
X_test_1 = X_test.copy()
X_test_1['banner_id'] = X_test_1['banner_id1']

In [122]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (12026120, 21), y_train shape: (12026120,)
X_test shape: (1883622, 21), y_test shape: (1883622,)


### 5. Кодирование категориальных признаков
Чтобы перевести категориальные фичи в числовые значения, воспользуемся One-hot encoding'ом, сопоставив каждой категории вектор из 0 с 1 на одной из позиций. Категориальными у нас являются все фичи, кроме одной -- campaign_clicks (числовая). К числовой фиче для нормализации и стандартизации применим StandartScaler.

In [123]:
numerical_columns = ['day', 'hour', 'minutes', 'rate0', 'g0', 'coeff_sum0', 'rate1', 'g1', 'coeff_sum1']
print('Numerical columns:')
print(numerical_columns)

# все остальные столбцы, кроме numerical columns и clicks
categorial_columns = ['zone_id', 'banner_id', 'os_id', 'country_id', 'banner_id0', 'banner_id1',
                      'zone_id:banner_id', 'zone_id:os_id', 'zone_id:country_id', 'banner_id:os_id',
                      'banner_id:country_id', 'os_id:country_id']
print('Categorial columns:')
print(categorial_columns)

Numerical columns:
['day', 'hour', 'minutes', 'rate0', 'g0', 'coeff_sum0', 'rate1', 'g1', 'coeff_sum1']
Categorial columns:
['zone_id', 'banner_id', 'os_id', 'country_id', 'banner_id0', 'banner_id1', 'zone_id:banner_id', 'zone_id:os_id', 'zone_id:country_id', 'banner_id:os_id', 'banner_id:country_id', 'os_id:country_id']


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack

# обработка категориальных фичей
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True, drop='first')

X_train_cat = oh_encoder.fit_transform(train_data[categorial_columns], y_train)
X_test_0_cat = oh_encoder.transform(X_test_0[categorial_columns])
X_test_1_cat = oh_encoder.transform(X_test_1[categorial_columns])

# обработка численных фичей
stand_scaler = StandardScaler()
X_train_num = stand_scaler.fit_transform(train_data[numerical_columns], y_train)
X_test_0_num = stand_scaler.transform(X_test_0[numerical_columns])
X_test_1_num = stand_scaler.transform(X_test_1[numerical_columns])

# соединение категориальных и численных фичей
X_train = hstack([X_train_cat, X_train_num])
X_test_0 = hstack([X_test_0_cat, X_test_0_num])
X_test_1 = hstack([X_test_1_cat, X_test_1_num])

### 6. Построение и обучение модели
В качестве модели выберем логистическую регрессию с 'liblinear' solver'ом. В ДЗ-1 уже нашли лучший коэффициент C=0.01.

In [125]:
from sklearn.linear_model import LogisticRegression

In [126]:
def create_train_model(X_train, y_train):
    return LogisticRegression(
        max_iter=100,
        solver='liblinear',
        C=0.01
    ).fit(X_train, y_train)


model = create_train_model(X_train, y_train)

### 7. Оценка качества обученной модели

In [127]:
from sklearn.metrics import roc_auc_score, log_loss


def calc_print_metrics(y_true, y_preds):
    print(f'Roc-auc baseline: {roc_auc_score(y_true, y_preds)}')
    print(f'Log-loss baseline: {log_loss(y_true, y_preds)}')

In [128]:
model_preds = model.predict_proba(X_test_0)[:, 1]
calc_print_metrics(y_test, model_preds)

Roc-auc baseline: 0.8064795952957515
Log-loss baseline: 0.13064818415025198


### 8. Расчет CIPS
Хотим посчитать OPE-метрику clipped ips на последнем дне. Знаем оптимальную лямбду: $\lambda = 10$.

Согласно условию, для определения победителя среди баннеров banner_id0 и banner_id1 проделывалось следующее:
1. Независимо подкидывались две нормальные величины -- $X_0$ и $X_1$:
В случае $\pi_0$: $N(coeff\_sum0, g0^2)$ и $N(coeff\_sum1, g1^2)$
В случае $\pi_1$: $N(coeff\_sum0_new, g0^2)$ и $N(coeff\_sum1_new, g1^2)$,
где $coeff\_sum0\_new$ и $coeff\_sum1\_new$ -- суммы коэффициентов для banner_id0 и banner_id1, которые предиктит обученная модель; их можно получить, применив logit функцию к вероятностям, которые предиктит обученная модель.

2. Выбирался тот баннер, чей семпл получался больше

Чтобы посчитать $\pi_i$ для каждого события нужно посчитать вероятность того, что одна нормальная величина больше другой.

Распишем вероятность того, что нормальная величина $X_0$ больше нормальной величины $X_1$: преобразуем неравенство, перенеся $X_1$ в левую часть
$$ P(X_0 > X_1) = P(X_0 - X_1 > 0) = $$

Получили 1 - определение функции распределения случайной величины $(X_0 - X_1)$ в точке 0: 
$$ = 1 - F_{X_0 - X_1}(0) $$

$X_0$ и $X_1$ являются независимыми нормальными величинами $\Rightarrow$ их разность $U = (X_1 - X_0)$ также является нормальной величиной. 
Матожидание разности: $E(X_0 - X_1) = E(X_0) - E(X_1)$ (линейность матожидания) 
Дисперсия разности: тк $X_0$ и $X_1$ независимы, дисперсия их разности равна сумме их дисперсий (св-во дисперсии): $D(X_0 - X_1) = D(X_0) + D(X_1)$

Итого:
$$ (X_0 - X_1) \sim N(E(X_0) - E(X_1), D(X_0) + D(X_1)) \Leftrightarrow $$
$$ (X_0 - X_1) \sim N(coeff\_sum0^* - coeff\_sum1^*, g0^2 + g1^2) $$

In [129]:
from scipy.stats import norm
from scipy.special import logit
import numpy as np

In [130]:
def calculate_policy(coeff_sum0, g0, coeff_sum1, g1):
    def f(x):
        return norm.cdf(x, loc=coeff_sum0 - coeff_sum1, scale=np.sqrt(g0 ** 2 + g1 ** 2))

    return 1 - f(0)

Найдем $\pi_0$:

In [ ]:
pi_0 = calculate_policy(
    test_data['coeff_sum0'],
    test_data['g0'],
    test_data['coeff_sum1'],
    test_data['g1']
)

Чтобы найти $\pi_1$, предварительно посчитаем $coeff\_sum0\_new$ и $coeff\_sum1\_new$, применив logit функцию к вероятностям, которые предиктит обученная модель:

In [132]:
y_preds0 = model.predict_proba(X_test_0)[:, 1]
coeff_sum0_new = logit(y_preds0)

y_preds1 = model.predict_proba(X_test_1)[:, 1]
coeff_sum1_new = logit(y_preds1)

Считаем $\pi_1$:

In [ ]:
pi_1 = calculate_policy(
    coeff_sum0_new,
    test_data['g0'],
    coeff_sum1_new,
    test_data['g1']
)

Считаем CIPS:
$$ V_{CIPS}(\pi, D_0) = \frac{1}{n} \sum_{i} r_i\ min[\frac{\pi(a_i | x_i)}{\pi_0(a_i | x_i)}, \lambda] $$

In [134]:
lambda_val = 10
# добавим небольшой eps в знаменатель дроби, чтобы не возникло деления на 0
cips = np.mean(y_test * np.minimum(pi_1 / (pi_0 + 1e-10), lambda_val))
print(f'CIPS: {cips}')

CIPS: 0.07431883358958129
